In [1]:
import pandas as pd

In [2]:
country_db = pd.read_csv('csvdir/Country-Code-Mapping.csv')
main_db = pd.read_csv('csvdir/HackerRank-Developer-Survey-2018-Numeric.csv')
question_db = pd.read_csv('csvdir/HackerRank-Developer-Survey-2018-Numeric-Mapping.csv')
codebook_db = pd.read_csv('csvdir/HackerRank-Developer-Survey-2018-Codebook.csv')

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,19,137,138,250) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#merge two tables by 'CountryNumeric2'
res = pd.merge(main_db,country_db,how='left',left_on=['CountryNumeric2'], right_on=['Value'])

#rename column
res = res.rename(columns={'Label':'CountryName'})

#convert NaN to -1
res['CountryName'] = res['CountryName'].fillna(-1)

#group by 'CountryName'
countries = res[['RespondentID','CountryName']].groupby('CountryName').count()
countries = countries.rename(columns={'RespondentID':'Count'})

#get count
russian_count = countries.filter(like='ussia',axis=0)
print(russian_count)
russian_count = russian_count.iat[0,0]

                    Count
CountryName              
Russian Federation    378


In [4]:
#slice by 'CountryName'
russian_developers = res[res['CountryName']=='Russian Federation']

#group by question about students
russian_students = russian_developers[['RespondentID','q8Student']].groupby('q8Student').count()

#slice by isStudent==True
russian_students = russian_students.filter(items=[1],axis=0)
russian_students = russian_students.iat[0,0]

#get percent
russian_students_percent = russian_students/russian_count
print(russian_students_percent*100)

12.962962962962962


In [5]:
#convert 'q9CurrentRole' from string to numeric
res = main_db
res[['q9CurrentRole']] = pd.to_numeric(res['q9CurrentRole'], errors='coerce')

#slice question_db by question=='q9CurrentRole'
res1 = question_db[question_db['Data Field'] == 'q9CurrentRole']

#merge it
res = pd.merge(res,res1,how='left',left_on=['q9CurrentRole'], right_on=['Value'])

#rename column
res = res.rename(columns={'Label':'RoleName'})

#group by 'RoleName'
res = res[['RespondentID','RoleName']].groupby('RoleName').count()
res = res.rename(columns={'RespondentID':'Count'})

#get counts
web_count = res.filter(like='Web', axis=0).iat[0,0]
data_count = res.filter(like='Data Scientist', axis=0).iat[0,0]
mobile_count = res.filter(like='Mobile', axis=0).iat[0,0]
all_count = main_db.shape[0]

#get percents
web_p=web_count/all_count
data_p=data_count/all_count
mobile_p=mobile_count/all_count

print(data_p*100, web_p*100, mobile_p*100)

1.9330410522120367 4.11319250697489 3.2363491430848943


In [6]:
codebook_db[codebook_db['Data Field'].isin(['q7Level1','q15Level2','q31Level3','q36Level4'])]

,Data Field,Survey Question,Notes
17,q7Level1,Success! You completed Level 1 of 4. To unloc...,right answer: 1 or C
50,q15Level2,You completed Level 2 of 4! To unlock the next...,"right answer: 1 or 'prints ""Hello World!"" n t..."
243,q31Level3,Woo hoo... you completed Level 3 of 4! To unlo...,right answer: 1 or num%2 == 0
250,q36Level4,Which of the following is useful in traversing...,right answer: 2 or Queue


In [7]:
#slice by right answers
res = russian_developers[(russian_developers['q7Level1'] == 1) &
                         (russian_developers['q15Level2'] == 1) &
                         (russian_developers['q31Level3'] == 1) &
                         (russian_developers['q36Level4'] == 2)]

#get count
absolute_right_count = res.shape[0]

#get percent
absolute_right_p = absolute_right_count/russian_developers.shape[0]

print(absolute_right_p*100)

50.52910052910053
